# Baseline

In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import tensorflow as tf
from keras import layers

import librosa

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import argparse
import wandb
from wandb.keras import WandbCallback

wandb.init(project="DACON_235905", name="Baseline")

parser = argparse.ArgumentParser(description="Baseline")
parser.add_argument('--sampling_rate', default=16000, type=int)
parser.add_argument('--learning_rate', default=0.01, type=float)
parser.add_argument('--batch_size', default=32, type=int)
parser.add_argument('--epochs', default=100, type=int)
parser.add_argument('--cv', default=10, type=int)
parser.add_argument('--seed', default=1011, type=int)
args = parser.parse_args('')

wandb.config.update(args)

sampling_rate=args.sampling_rate
learning_rate=args.learning_rate
BATCH_SIZE = args.batch_size
EPOCHS = args.epochs
cv = args.cv
seed = args.seed

def set_seeds(seed=seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
set_seeds()

train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

train.head()

wandb: Currently logged in as: gnoeyheat (use `wandb login --relogin` to force relogin)


,file_name,label
0,001.wav,9
1,002.wav,0
2,004.wav,1
3,005.wav,8
4,006.wav,0


## Preprocessing

In [2]:
train_folder = "data/train/"
test_folder = "data/test/"

def dataset(folder, mode):
    dataset = []
    for file in tqdm(os.listdir(folder)):
        if 'wav' in file:
            abs_file_path = os.path.join(folder, file)
            data, sr = librosa.load(abs_file_path, sr=sampling_rate)
            data = librosa.util.normalize(data)
            if mode == "train":
                class_label = int(train[train.file_name == file].label)
                dataset.append([data, file, class_label])
            elif mode == "test":
                dataset.append([data, file])
    if mode == "train":
        dataset = pd.DataFrame(dataset, columns=['data', 'file_name', 'label'])
    if mode == "test":
        dataset = pd.DataFrame(dataset, columns=['data', 'file_name'])
    return dataset

train_df = dataset(train_folder, "train")
test_df = dataset(test_folder, "test")

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:05<00:00, 35.56it/s]


In [3]:
def preprocess_dataset(data):

    frame_length = 0.025
    frame_stride = 0.010

    input_nfft = int(round(sampling_rate*frame_length))
    input_stride = int(round(sampling_rate*frame_stride))

    mel_s = []
    for i in tqdm(data):
        extracted_features = librosa.feature.mfcc(y=i,
                                                  sr=sampling_rate,
                                                  n_mfcc=40,
                                                  n_fft=input_nfft,
                                                  hop_length=input_stride)
        extracted_features = tf.image.resize(extracted_features[:, :, np.newaxis], (40, 40))
        mel_s.append(extracted_features)
    return mel_s

X = np.array(preprocess_dataset(train_df["data"]))
X_test = np.array(preprocess_dataset(test_df["data"]))
y = train_df["label"]

X.shape, y.shape, X_test.shape

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 207.77it/s]


((400, 40, 40, 1), (400,), (200, 40, 40, 1))

## Training

In [4]:
def CNN():
    inp = tf.keras.Input(shape=(X.shape[1], X.shape[2], 1))
    x = layers.Conv2D(32, 3, 1, padding="same")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(256, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation="relu")(x)
    oup = layers.Dense(len(np.unique(y)), activation="softmax")(x)
    model = tf.keras.Model(inputs=inp, outputs=oup)
    return model

model = CNN()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 40, 40, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 40, 40, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 40, 40, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 40, 40, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 20, 20, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 64)        18496 

In [5]:
validations = []
predictions = []

idx=0

skf = StratifiedKFold(n_splits=cv)
for train_index, val_index in tqdm(skf.split(X, y)):
    
    idx+=1

    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    train_ds = (
        tf.data.Dataset.from_tensor_slices((X_train, y_train))
        .shuffle(len(X_train))
        .batch(BATCH_SIZE)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    val_ds = (
        tf.data.Dataset.from_tensor_slices((X_val, y_val))
        .batch(BATCH_SIZE)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    model = CNN()

    lr = tf.keras.optimizers.schedules.CosineDecay(learning_rate, decay_steps=1000)
    optim = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)

    model.compile(
        optimizer=optim,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    
    checkpoint_filepath=f"load_model/{parser.description}_{idx}"

    checkpoint_callback = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
        ),
        tf.keras.callbacks.ModelCheckpoint(
            checkpoint_filepath,
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True
        )
    ]

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[checkpoint_callback, WandbCallback()],
    )

    validations.append(accuracy_score(y_val, np.argmax(model.predict(X_val), axis=1)))
    predictions.append(model.predict(X_test))

val_acc = np.mean(validations, axis=0)

print("validation_accuracy: ", val_acc)
wandb.log({'validation_accuracy': val_acc})

0it [00:00, ?it/s]

Epoch 1/100
12/12 [==============================] - 6s 74ms/step - loss: 2.1717 - accuracy: 0.2528 - val_loss: 2.2337 - val_accuracy: 0.1500 - _timestamp: 1655493483.0000 - _runtime: 38.0000
Epoch 2/100
12/12 [==============================] - 0s 39ms/step - loss: 1.6552 - accuracy: 0.6111 - val_loss: 2.0820 - val_accuracy: 0.4000 - _timestamp: 1655493484.0000 - _runtime: 39.0000
Epoch 3/100
12/12 [==============================] - 0s 36ms/step - loss: 1.1383 - accuracy: 0.7389 - val_loss: 1.8376 - val_accuracy: 0.1750 - _timestamp: 1655493484.0000 - _runtime: 39.0000
Epoch 4/100
12/12 [==============================] - 0s 21ms/step - loss: 0.7166 - accuracy: 0.8806 - val_loss: 1.9127 - val_accuracy: 0.3500 - _timestamp: 1655493484.0000 - _runtime: 39.0000
Epoch 5/100
12/12 [==============================] - 0s 38ms/step - loss: 0.5180 - accuracy: 0.9028 - val_loss: 1.2393 - val_accuracy: 0.7250 - _timestamp: 1655493485.0000 - _runtime: 40.0000
Epoch 6/100
12/12 [=====================

1it [00:24, 24.38s/it]

Epoch 1/100
12/12 [==============================] - 1s 45ms/step - loss: 2.1664 - accuracy: 0.3194 - val_loss: 2.2589 - val_accuracy: 0.1750 - _timestamp: 1655493502.0000 - _runtime: 57.0000
Epoch 2/100
12/12 [==============================] - 0s 30ms/step - loss: 1.6503 - accuracy: 0.6250 - val_loss: 2.0111 - val_accuracy: 0.1500 - _timestamp: 1655493503.0000 - _runtime: 58.0000
Epoch 3/100
12/12 [==============================] - 0s 31ms/step - loss: 1.1414 - accuracy: 0.7417 - val_loss: 1.7727 - val_accuracy: 0.3000 - _timestamp: 1655493503.0000 - _runtime: 58.0000
Epoch 4/100
12/12 [==============================] - 0s 29ms/step - loss: 0.7668 - accuracy: 0.8611 - val_loss: 1.7001 - val_accuracy: 0.3500 - _timestamp: 1655493504.0000 - _runtime: 59.0000
Epoch 5/100
12/12 [==============================] - 0s 22ms/step - loss: 0.5345 - accuracy: 0.8778 - val_loss: 1.8085 - val_accuracy: 0.4500 - _timestamp: 1655493504.0000 - _runtime: 59.0000
Epoch 6/100
12/12 [=====================

2it [00:40, 19.47s/it]

...

Epoch 1/100
12/12 [==============================] - 1s 47ms/step - loss: 2.2830 - accuracy: 0.2222 - val_loss: 2.2687 - val_accuracy: 0.1500 - _timestamp: 1655493611.0000 - _runtime: 166.0000
Epoch 2/100
12/12 [==============================] - 0s 30ms/step - loss: 1.8808 - accuracy: 0.4889 - val_loss: 2.0781 - val_accuracy: 0.2000 - _timestamp: 1655493612.0000 - _runtime: 167.0000
Epoch 3/100
12/12 [==============================] - 0s 31ms/step - loss: 1.4184 - accuracy: 0.6250 - val_loss: 1.8937 - val_accuracy: 0.3250 - _timestamp: 1655493612.0000 - _runtime: 167.0000
Epoch 4/100
12/12 [==============================] - 0s 31ms/step - loss: 0.9478 - accuracy: 0.7889 - val_loss: 1.5682 - val_accuracy: 0.5500 - _timestamp: 1655493613.0000 - _runtime: 168.0000
Epoch 5/100
12/12 [==============================] - 0s 29ms/step - loss: 0.5974 - accuracy: 0.8917 - val_loss: 1.2628 - val_accuracy: 0.7500 - _timestamp: 1655493613.0000 - _runtime: 168.0000
Epoch 6/100
12/12 [================

9it [02:33, 17.97s/it]

Epoch 1/100
12/12 [==============================] - 1s 47ms/step - loss: 2.2243 - accuracy: 0.1889 - val_loss: 2.3139 - val_accuracy: 0.2000 - _timestamp: 1655493631.0000 - _runtime: 186.0000
Epoch 2/100
12/12 [==============================] - 0s 30ms/step - loss: 1.7378 - accuracy: 0.6139 - val_loss: 2.1889 - val_accuracy: 0.2750 - _timestamp: 1655493632.0000 - _runtime: 187.0000
Epoch 3/100
12/12 [==============================] - 0s 29ms/step - loss: 1.2098 - accuracy: 0.8028 - val_loss: 1.8807 - val_accuracy: 0.5000 - _timestamp: 1655493632.0000 - _runtime: 187.0000
Epoch 4/100
12/12 [==============================] - 0s 30ms/step - loss: 0.7538 - accuracy: 0.8806 - val_loss: 1.5035 - val_accuracy: 0.5500 - _timestamp: 1655493633.0000 - _runtime: 188.0000
Epoch 5/100
12/12 [==============================] - 0s 30ms/step - loss: 0.4558 - accuracy: 0.9194 - val_loss: 1.2827 - val_accuracy: 0.6500 - _timestamp: 1655493633.0000 - _runtime: 188.0000
Epoch 6/100
12/12 [================

10it [02:56, 17.64s/it]

validation_accuracy:  0.975


## Inference

In [6]:
test_df["label"] = np.argmax(np.mean(predictions, axis=0), axis=1)
test_df = test_df.sort_values("file_name").reset_index(drop=True)
test_df.head()

,data,file_name,label
0,"[-0.0037829715, -0.00676741, -0.006659494, -0....",003.wav,0
1,"[0.026130762, 0.045480434, 0.040787034, 0.0447...",008.wav,9
2,"[0.01041246, 0.017926143, 0.0169886, 0.0169302...",010.wav,3
3,"[-0.0018547748, -0.0028658605, -0.0018936209, ...",015.wav,8
4,"[0.014641531, 0.021667747, 0.013923713, 0.0177...",024.wav,2


In [7]:
submission = pd.read_csv('data/sample_submission.csv')
submission['label'] = test_df['label']
submission.to_csv('submission.csv', index=False)

submission.head()

,file_name,label
0,003.wav,0
1,008.wav,9
2,010.wav,3
3,015.wav,8
4,024.wav,2


In [8]:
answer = pd.read_csv('answer.csv')["label"]
test_acc = accuracy_score(answer, test_df["label"])

print("test_accuracy: ", test_acc)
wandb.log({'test_accuracy': test_acc})

test_accuracy:  1.0


# 